# Load DICOM data stored in IGT Cloud


The url to the cloud:

In [1]:
igtcloud_domain = 'igt-web.eu1.phsdp.com'
username = 'danny.ruijters@philips.com'
# username = '<your hsdp iam username>'

Import the igtcloud packages:

In [3]:
import logging
logging.basicConfig(level=logging.INFO)

from igtcloud.client.services import entities_service, auth_service
from igtcloud.client.core.auth import AuthRefresher

Either login using `igtcloud login` or use the following cell

In [4]:
_auth_refresher = AuthRefresher(domain=igtcloud_domain, username=username)
_auth_handler = _auth_refresher.__enter__()
entities_service.auth_handler = _auth_handler
auth_service.auth_handler = _auth_handler

INFO:igtcloud.client.core.auth:Starting IGT Cloud authentication handler...
INFO:igtcloud.client.core.auth:Login to IGT Cloud (igt-web.eu1.phsdp.com)


danny.ruijters@philips.com@igt-web.eu1.phsdp.com's password:  ········


INFO:igtcloud.client.core.auth:Logged in as: danny.ruijters@philips.com


List all institutes (hospitals) in your account:

In [5]:
projects = entities_service.get_projects()

import ipywidgets as widgets
select_proj = widgets.Select(
    options=[i.description for i in projects],
    rows=10, layout=widgets.Layout(width="auto"),
    description='Projects:'
)
select_proj

Select(description='Projects:', layout=Layout(width='auto'), options=('Istanbul', 'AD Open Innovation project'…

In [6]:
project = projects[select_proj.index]
institutes = entities_service.get_institutes(project.id)

select_inst = widgets.Select(
    options=[i.description for i in institutes],
    rows=10, layout=widgets.Layout(width="auto"),
    description='Institutes:'
)
select_inst

Select(description='Institutes:', layout=Layout(width='auto'), options=('Medusa - Philips IGT annotation tool …

Load an overview of the studies for the selected hospitals, and list the patient names:

In [7]:
institute = institutes[select_inst.index]
#print(institute)
studies = entities_service.get_studies(institute.id)

select_pat = widgets.Select(
    options=[s.patient.patient_name for s in studies],
    rows=10, layout=widgets.Layout(width="auto"),
    description='Patients:'
)
select_pat

Select(description='Patients:', layout=Layout(width='auto'), options=('Medusa-annotate-00002', 'Medusa-annotat…

Display all the thumbnail preview images in the selected patient:

In [8]:
# display the thumbnails
import io
from IPython.display import display

def preview(files):
    images = []
    for file in files[:50]:  #parse first 50 files to keep processing time limited
        if file.type == '.jpg' or file.type == '.png':
            with io.BytesIO() as fp:
                file.download_fileobj(fp)
                fp.seek(0)
                img = widgets.Image(value=fp.read(), format=file.type[1:])
                if len(images) > 0: img.height=images[0].height
                images.append(img)
    display(widgets.HBox(images))

study_id = studies[select_pat.index].study_database_id
study_files = entities_service.get_study_files(institute.id, study_id, auxiliary=True)
if hasattr(study_files, 'dicom'): preview(study_files.dicom)
if hasattr(study_files, 'files'): preview(study_files.files)

Select one of the files:

In [9]:
if hasattr(study_files, 'dicom'):
    dicom_files = [file for file in study_files.dicom if file.type == 'unknown']
else:
    dicom_files = [file for file in study_files.files if file.type == 'unknown']
    
select_file1 = widgets.Select(
    options=[f.file_name for f in dicom_files],
    rows=10, layout=widgets.Layout(width="auto"),
    description='Files:'
)
select_file2 = widgets.Select(
    options=[f.file_name for f in dicom_files],
    rows=10, layout=widgets.Layout(width="auto"),
    description='Files:'
)
select_file3 = widgets.Select(
    options=[f.file_name for f in dicom_files],
    rows=10, layout=widgets.Layout(width="auto"),
    description='Files:'
)
select_file1

Select(description='Files:', layout=Layout(width='auto'), options=('IVUS/20230731171358070b5e8639efb/DICOMDIR'…

In [10]:
select_file2

Select(description='Files:', layout=Layout(width='auto'), options=('IVUS/20230731171358070b5e8639efb/DICOMDIR'…

In [11]:
select_file3

Select(description='Files:', layout=Layout(width='auto'), options=('IVUS/20230731171358070b5e8639efb/DICOMDIR'…

Download the selected file, store as temporary file, and interpret as DICOM:

In [14]:
import tempfile
import pydicom

def LoadDicom(file):
    with tempfile.TemporaryFile() as fp:
        file.download_fileobj(fp)
        fp.seek(0)
        return pydicom.read_file(fp)

dicom1 = LoadDicom(dicom_files[select_file1.index])
dicom2 = LoadDicom(dicom_files[select_file2.index])
dicom3 = LoadDicom(dicom_files[select_file3.index])

View the downloaded DICOM file:

In [15]:
import cv2
from ipywidgets import interact, widgets

def jpeg(dicom, perc):
    idx =  perc * (len(dicom.pixel_array)-1) // 1000
    img = dicom.pixel_array[idx]
    resized = cv2.resize(img, (300, 300), interpolation = cv2.INTER_AREA)
    success, jpegbuffer = cv2.imencode(".jpg", resized)
    return jpegbuffer

def dicom_animation2(n):
    images[0].value = jpeg(dicom1, n)
    images[1].value = jpeg(dicom2, n)
    images[2].value = jpeg(dicom3, n)
    return n

images = [ widgets.Image(), widgets.Image(), widgets.Image() ]
display(widgets.HBox(images))
interact(dicom_animation2, n=(0, 1000))

interactive(children=(IntSlider(value=500, description='n', max=1000), Output()), _dom_classes=('widget-intera…

<function __main__.dicom_animation2(n)>